In [ ]:
!pip install transformers datasets wandb
!pip install accerate -U

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.8/313.8 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.2 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement accerate (from versions: none)
ERROR: No matching distribution found for accerate


In [ ]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
import csv
from datasets import Dataset, DatasetDict

def csv_to_dataset(path, do_train_test_split=True):
    data = []
    with open(path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            # Replace 'positive' with 1 and 'negative' with 0
            row['Sentiment'] = 1 if row['Sentiment'] == 'POSITIVE' else 0
            data.append(row)

    # Convert the list of dictionaries to a Hugging Face Dataset
    dataset = Dataset.from_dict({key: [d[key] for d in data] for key in data[0]})

    # Rename columns
    dataset = dataset.rename_column("Sentiment", "label")
    dataset = dataset.rename_column("Text", "text")

    # Train test split
    if do_train_test_split:
        dataset = dataset.train_test_split(test_size=0.2, seed=42)
        # rename splits test -> val
        dataset = DatasetDict({
                    "train": dataset["train"],
                    "val": dataset["test"]})


    return dataset

In [ ]:
from transformers import AutoTokenizer
from transformers import TrainingArguments, Trainer
from transformers import DataCollatorWithPadding
import numpy as np
from transformers import AutoModelForSequenceClassification
import torch
import os


print("torch version: ", torch.__version__)
if torch.cuda.is_available():
    print("CUDA is available. PyTorch is using GPU.")
else:
    print("CUDA is not available. PyTorch is using CPU.")

def set_wandb_mode(mode):
    os.environ["WANDB_MODE"] = mode

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    N = predictions.shape[0]
    accuracy = (labels == predictions).sum() / N
    return {"accuracy": accuracy}

def train_and_evaluate_models(model_names, train_dataset_path, test_dataset_path, wandb_logging=True, make_val_split=True, wandb_group=None):
    """
    :param model_names:
    :type model_names:
    :param train_dataset_path:
    :type train_dataset_path:
    :return:
    :rtype:
    """

    # build dataset
    if make_val_split:
      train_dataset = csv_to_dataset(train_dataset_path)
    else:
      train_dataset = csv_to_dataset(train_dataset_path, do_train_test_split=False)
    test_dataset = csv_to_dataset(test_dataset_path, do_train_test_split=False)


    for model_name in model_names:
        if isinstance(model_name, str):
          tokenizer_name = model_name
        else:
          tokenizer_name = model_name["tokenizer_name"]
          model_name = model_name["model_name"]
        tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
        model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

        def tokenize(examples):
          return tokenizer(examples["text"], truncation=True)

        # Tokenize Dataset
        if make_val_split:
          tokenized_train = train_dataset["train"].map(tokenize, batched=True)
          tokenized_val = train_dataset["val"].map(tokenize, batched=True)
          tokenized_test = test_dataset.map(tokenize, batched=True)
        else:
          tokenized_train = train_dataset.shuffle(seed=42).map(tokenize, batched=True)
          tokenized_val = test_dataset.map(tokenize, batched=True)

        data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
        # Define training arguments
        training_args = TrainingArguments(
            output_dir=f"./results/{model_name}",
            eval_strategy="steps",
            eval_steps = 0.05,
            save_steps = 0.05,
            logging_dir=f"./logs/{model_name}",
            logging_steps=10,
            report_to="wandb",
            learning_rate = 1e-4,
            run_name=model_name,
            save_total_limit=3,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            num_train_epochs=5,
            weight_decay=0.01,
            load_best_model_at_end = True,
            metric_for_best_model = "eval_accuracy",
            greater_is_better = True
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            data_collator=data_collator,
            train_dataset=tokenized_train,
            eval_dataset=tokenized_val,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics
        )

        model_name = model_name.split(sep='/')[-1].replace(':', '_')

        if wandb_logging:
          wandb.init(project="ChungliAoSentiment", name=model_name.replace("/", "-"), group=wandb_group)

        trainer.train()
        if make_val_split:
          # reset trainer to prevent test results being logged to eval results on wandb
          training_args = TrainingArguments(
            output_dir=f"./results/{model_name}",
            evaluation_strategy="epoch",
            logging_dir=f"./logs/{model_name}",
            logging_steps=10,
            report_to="none",
            run_name=model_name,
            save_total_limit=3,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            num_train_epochs=5,
            weight_decay=0.01,
          )

          trainer = Trainer(
            model=model,
            args=training_args,
            data_collator=data_collator,
            train_dataset=tokenized_train,
            eval_dataset=tokenized_val,
            tokenizer=tokenizer,
            compute_metrics=compute_metrics
          )


          test_metrics = trainer.evaluate(eval_dataset=tokenized_test)
          print(test_metrics)
          #replace "eval" with "test" in all dict keys
          test_metrics = {key.replace("eval", "test"): value for key, value in test_metrics.items()}
          #Log test results to W&B summary
          for key, value in test_metrics.items():
            wandb.run.summary[key] = value

        save_path = model_name
        trainer.save_model(save_path)
        # Log the checkpoint as an artifact
        artifact = wandb.Artifact(name="Chungliao-xlm-roberta-sentiment", type="model")
        artifact.add_dir(save_path)
        wandb.log_artifact(artifact)


torch version:  2.4.1+cu121
CUDA is available. PyTorch is using GPU.


In [ ]:
model_list = ["google-bert/bert-base-multilingual-cased", "N1ch0/chungliao-mbert-base-cased", "FacebookAI/xlm-roberta-base", "N1ch0/chungliao-xlm-roberta-base", "robzchhangte/MizBERT", "N1ch0/chungliao-MizoBert"]
train_dataset_path = "/content/Chungli_Ao_train_set.csv"
test_data_path = "/content/Test_data_Chungli_ao.csv"

In [ ]:
train_and_evaluate_models(model_list, train_dataset_path, test_data_path, make_val_split=False)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

...

Map:   0%|          | 0/8579 [00:00<?, ? examples/s]

Map:   0%|          | 0/4095 [00:00<?, ? examples/s]

eval/accuracy,▁▂▃▅▅▆▆▆▃▇█▅▇▁▆▆▅▅▆
eval/loss,▇▃█▁▅▃▃▂▂▁▁▄▁▆▂▅▄▂▂
eval/runtime,█▅▆▃▅▄▅▆▆▃▂▄▃▃▁▃▁▄▂
eval/samples_per_second,▁▄▃▆▄▅▄▃▃▆▇▅▆▆█▆█▅▇
eval/steps_per_second,▁▄▃▆▄▅▄▃▃▆▇▅▆▆█▆█▅▇
train/epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇█
train/global_step,▁▁▁▂▂▂▂▃▃▃▃▄▄▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▆▆▆▆▇▇▇▇███
train/grad_norm,▁▂▁▂▁▁▃▁▁█▂▁▁▂▁▃▁▁▁▂▁▂▁▁▁▁▁▁▁▁▁▅▁▁▁▁▂▁▁▂
train/learning_rate,█████▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▁
train/loss,▄█▃▅▃▂▂▃▃▂▃▃▁▂▂▁▂▄▄▂▃▂▂▂▃▃▃▂▂▃▂▂▂▁▃▂▂▃▂▄
eval/accuracy,0.76386


Step,Training Loss,Validation Loss,Accuracy
135,0.149800,0.566932,0.780708
270,0.226300,0.695057,0.767766
405,0.078500,0.627159,0.785348
540,0.390100,0.727084,0.791209
675,0.110300,0.632607,0.813431
810,0.205000,0.614096,0.821490
945,0.061900,0.857664,0.774603
1080,0.089200,0.621977,0.807814
1215,0.217700,0.595664,0.797314
1350,0.201400,0.664943,0.796825


wandb: Adding directory to artifact (./chungliao-MizoBert)... Done. 1.8s
